In [ ]:
import matplotlib.pyplot as plt
from morse_helpers import convert_image_to_array
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape, Conv2D, Conv1D, MaxPooling2D, Input, AveragePooling2D, MaxPooling1D, ZeroPadding2D
import tensorflow as tf
import numpy as np

In [ ]:
# Retrieve saved model
loaded_model = tf.keras.models.load_model("saved_model")

# Retrieve saved model
loaded_model_2 = tf.keras.models.load_model("saved_model_2")

In [ ]:
def show_image(img):
    fig = plt.figure(figsize=(30,5))
    plt.xlim(0, 700)
    plt.imshow(img)
  
    return fig

In [ ]:
def get_noisy_image(img, noise_level):
    tf.keras.backend.set_learning_phase(1)
    x = tf.reshape(img, [1, 5, 1400, 1])
    x = tf.keras.layers.GaussianNoise(noise_level)(x)
    return x[0]

def get_img_prediction(img, model):
    img_exp = np.expand_dims(img, axis=0)
    pred = model.predict(img_exp)
    return int(pred[0][0][0] * 1400), np.argmax(pred[1])

def print_prediction(prediction, description):
    print("prediction on image ", description, " : position, letter :", prediction[0], prediction[1])


In [ ]:
# Add noise to image and predict and display
orig_img = convert_image_to_array("wordsMatrices_13_220/1.png", (5, 1400))

NOISE_LEVELS = [0.1, 0.2, 0.3]

for level in NOISE_LEVELS:
    noisy_img_01 = get_noisy_image(orig_img, level)

    show_image(noisy_img_01)
    print_prediction(get_img_prediction(noisy_img_01, loaded_model), "original image")
    plt.show()


In [ ]:
def get_orig_image_arrays(amount):
    orig_images = []
    for i in range(amount):
        orig_images.append(convert_image_to_array("wordsMatrices_13_220/" + str(i) + ".png", (5, 1400)))

    return orig_images

In [ ]:
def test_moel_noise_level(noise_level, amount, model, orig_images):
    counter = 0
    for i in range(amount):
        orig_img = orig_images[i]
        orig_pred = get_img_prediction(orig_img, model)
        noisey_pred = get_img_prediction(get_noisy_image(orig_img, noise_level), model)
        if orig_pred[1] != noisey_pred[1]:
            counter += 1
    print("for noise level", noise_level, ":", counter/amount * 100, "% wrong", "model name", model.name, "amount", amount)
    return counter/amount


In [ ]:
AMOUNT = 500
orig_images = get_orig_image_arrays(AMOUNT)

In [ ]:
MODELS = [loaded_model, loaded_model_2]

for level in NOISE_LEVELS:
    for model in MODELS:
        performance = test_moel_noise_level(level, AMOUNT, model, orig_images)
